INSPER - Big Data e Computação em Nuvem

# Big Data e Computação em Nuvem - Projeto Final: Análise de Atraso de Voos

**Integrantes**:
- Flávio
- Hélio
- Michel

In [2]:
#Importando as bibliotecas necessárias
import seaborn as sns
import pyspark.sql.functions as f
from pyspark.sql.types import StringType
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, when, stddev, mean, count, isnan
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Correlation

In [3]:
# Criar a sessao do Spark

spark = SparkSession \
            .builder \
            .master("local[4]") \
            .appName("nyc_<mudar-nome>") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
            .getOrCreate()

In [4]:
STORAGE_ACCOUNT = 'dlspadseastusprod'
CONTAINER = 'big-data-comp-nuvem'
FOLDER = 'airline-delay'
TOKEN = 'lSuH4ZI9BhOFEhCF/7ZQbrpPBIhgtLcPDfXjJ8lMxQZjaADW4p6tcmiZGDX9u05o7FqSE2t9d2RD+ASt0YFG8g=='

spark.conf.set("fs.azure.account.key." + STORAGE_ACCOUNT + ".blob.core.windows.net", TOKEN)

In [11]:
# Vamos usar os dados de 2012
df = spark.read.csv("wasbs://{}@{}.blob.core.windows.net/{}/2012.csv".format(CONTAINER, STORAGE_ACCOUNT, FOLDER), header=True, inferSchema=True)

- Definir o escopo do negócio: qùal será a definição do atraso?
- A análise toda é do ponto do vista da empresa, consumidor ou governo (regulador)?
- Criar variavel resposta
- Tratamento de missing values
- Verificar a qualidade das informações (estao no tipo certo?)
- Criar colunas novas? Vamos manter todas as colunas para a etapa de modelagem?
- Vamos usar outras variaveis (preço de petroleo, pois QAV representa 41% do custo em 2022, segundo a ANAC)? Como era o desempenho economico em cada ano?
- Faremos filtros para retirar anos outliers?
- Vamos retirar outliers? Verificar anos atípicos
- verificar a influência de variáveis externa (desempenho economico, preço do petróleo, tempo)
- Matriz de correlação
- Analise bivariada: % atrasos por aeroporto, por regiao


### Definir o escopo do negócio: qùal será a definição do atraso?

O atraso será definido a partir da soma das coluna DEP_DELAY e ARR_DELAY. Sem tolerência de atraso (DEP_DELAY + ARR_DELAY > 0).

In [7]:
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: double (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: d

In [10]:
df.limit(5).toPandas()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,XE,1204,DCA,EWR,1100.0,1058.0,-2.0,18.0,1116.0,...,62.0,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,None
1,2009-01-01,XE,1206,EWR,IAD,1510.0,1509.0,-1.0,28.0,1537.0,...,82.0,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,None
2,2009-01-01,XE,1207,EWR,DCA,1100.0,1059.0,-1.0,20.0,1119.0,...,70.0,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,None
3,2009-01-01,XE,1208,DCA,EWR,1240.0,1249.0,9.0,10.0,1259.0,...,77.0,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,None
4,2009-01-01,XE,1209,IAD,EWR,1715.0,1705.0,-10.0,24.0,1729.0,...,105.0,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,None


In [35]:
#Remover a coluna Unnamed
df = df.drop("Unnamed: 27")
df.limit(5).toPandas()

/opt/conda/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,DELAY,DISTANCE_TYPE
0,2012-01-01,MQ,4041,BMI,ORD,1715.0,1702.0,-13.0,9.0,1711.0,...,37.0,116.0,NaN,NaN,NaN,NaN,NaN,None,0,near
1,2012-01-01,MQ,4041,ORD,BMI,1545.0,1540.0,-5.0,11.0,1551.0,...,23.0,116.0,NaN,NaN,NaN,NaN,NaN,None,0,near
2,2012-01-01,MQ,4042,SGF,ORD,715.0,705.0,-10.0,12.0,717.0,...,75.0,438.0,NaN,NaN,NaN,NaN,NaN,None,0,near
3,2012-01-01,MQ,4045,ORD,DBQ,1535.0,1529.0,-6.0,13.0,1542.0,...,31.0,147.0,NaN,NaN,NaN,NaN,NaN,None,0,near
4,2012-01-01,MQ,4047,ORD,RIC,1600.0,1615.0,15.0,12.0,1627.0,...,86.0,642.0,NaN,NaN,NaN,NaN,NaN,None,1,medium


In [9]:
# numero de linhas
total_rows = df.count()

In [10]:
total_rows

6096762

In [11]:
#Verificando os missing values
{col:df.filter(df[col].isNull()).count() for col in df.columns}

{'FL_DATE': 0,
 'OP_CARRIER': 0,
 'OP_CARRIER_FL_NUM': 0,
 'ORIGIN': 0,
 'DEST': 0,
 'CRS_DEP_TIME': 1,
 'DEP_TIME': 75723,
 'DEP_DELAY': 75723,
 'TAXI_OUT': 77977,
 'WHEELS_OFF': 77977,
 'WHEELS_ON': 80894,
 'TAXI_IN': 80894,
 'CRS_ARR_TIME': 2,
 'ARR_TIME': 80894,
 'ARR_DELAY': 91381,
 'CANCELLED': 0,
 'CANCELLATION_CODE': 6017900,
 'DIVERTED': 0,
 'CRS_ELAPSED_TIME': 1,
 'ACTUAL_ELAPSED_TIME': 91381,
 'AIR_TIME': 91381,
 'DISTANCE': 0,
 'CARRIER_DELAY': 5081604,
 'WEATHER_DELAY': 5081604,
 'NAS_DELAY': 5081604,
 'SECURITY_DELAY': 5081604,
 'LATE_AIRCRAFT_DELAY': 5081604,
 'Unnamed: 27': 6096762}

In [21]:
#tratar os missing values (inicialmente coloquei para excluir todas as linhas que tem missing values)
df = df.na.drop(subset=['DEP_DELAY','TAXI_OUT', 'TAXI_IN', 'ARR_DELAY', 'CRS_ELAPSED_TIME', 'AIR_TIME', 'ACTUAL_ELAPSED_TIME'])

In [22]:
#Verificando se as linhas com missing values foram excluiídas
{col:df.filter(df[col].isNull()).count() for col in df.columns}

{'FL_DATE': 0,
 'OP_CARRIER': 0,
 'OP_CARRIER_FL_NUM': 0,
 'ORIGIN': 0,
 'DEST': 0,
 'CRS_DEP_TIME': 0,
 'DEP_TIME': 0,
 'DEP_DELAY': 0,
 'TAXI_OUT': 0,
 'WHEELS_OFF': 0,
 'WHEELS_ON': 0,
 'TAXI_IN': 0,
 'CRS_ARR_TIME': 1,
 'ARR_TIME': 0,
 'ARR_DELAY': 0,
 'CANCELLED': 0,
 'CANCELLATION_CODE': 6005380,
 'DIVERTED': 0,
 'CRS_ELAPSED_TIME': 0,
 'ACTUAL_ELAPSED_TIME': 0,
 'AIR_TIME': 0,
 'DISTANCE': 0,
 'CARRIER_DELAY': 4990223,
 'WEATHER_DELAY': 4990223,
 'NAS_DELAY': 4990223,
 'SECURITY_DELAY': 4990223,
 'LATE_AIRCRAFT_DELAY': 4990223,
 'Unnamed: 27': 6005380}

In [18]:
#exibir as principais estatísticas descritivas do conjunto de dados.
df.describe().show()

+-------+----------+-----------------+-------+-------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+-----------------+--------------------+-----------------+-------------------+------------------+-----------------+-----------------+------------------+------------------+-------------------+-------------------+-----------+-------------+
|summary|OP_CARRIER|OP_CARRIER_FL_NUM| ORIGIN|   DEST|      CRS_DEP_TIME|         DEP_TIME|        DEP_DELAY|          TAXI_OUT|        WHEELS_OFF|         WHEELS_ON|          TAXI_IN|      CRS_ARR_TIME|          ARR_TIME|         ARR_DELAY|          CANCELLED|CANCELLATION_CODE|            DIVERTED| CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|          AIR_TIME|         DISTANCE|    CARRIER_DELAY|     WEATHER_DELAY|         NAS_DELAY|     SECURITY_DELAY|LATE_AIRCRAFT_DELAY|Unnamed: 27|DISTANCE_TYPE|
+---

In [23]:
#Criar a variável reposta a partir da soma das colunas ARR_DELAY DEP_DELAY
df = df.withColumn("DELAY", 
                   when(df['ARR_DELAY'] +  df['DEP_DELAY'] > 0, 1)
                   .otherwise(0))

In [ ]:
#Verificar a assimetria das variáveis?

In [32]:
# Verificar se há valores negativos
def conta_negativo(coluna):
    try: 
        return df.filter((df[coluna]<0)).count()
    except:
        return "Variável não numérica"

{col:conta_negativo(col) for col in df.columns}

{'FL_DATE': 'Variável não numérica',
 'OP_CARRIER': 0,
 'OP_CARRIER_FL_NUM': 0,
 'ORIGIN': 0,
 'DEST': 0,
 'CRS_DEP_TIME': 0,
 'DEP_TIME': 0,
 'DEP_DELAY': 3478053,
 'TAXI_OUT': 0,
 'WHEELS_OFF': 0,
 'WHEELS_ON': 0,
 'TAXI_IN': 0,
 'CRS_ARR_TIME': 0,
 'ARR_TIME': 0,
 'ARR_DELAY': 3735412,
 'CANCELLED': 0,
 'CANCELLATION_CODE': 0,
 'DIVERTED': 0,
 'CRS_ELAPSED_TIME': 0,
 'ACTUAL_ELAPSED_TIME': 0,
 'AIR_TIME': 0,
 'DISTANCE': 0,
 'CARRIER_DELAY': 0,
 'WEATHER_DELAY': 0,
 'NAS_DELAY': 0,
 'SECURITY_DELAY': 0,
 'LATE_AIRCRAFT_DELAY': 0,
 'Unnamed: 27': 0,
 'DELAY': 0,
 'DISTANCE_TYPE': 0}

### Criar novas colunas

A partir do histograma acima, vamos definir o seguinte critério para classificar as distâncias:

- próximo: abaixo de 500 milhas
- médio: entre 500 e 1000 milhas
- distante: acima de 1000 milhas

In [25]:
#Criar coluna "Tipo de Distância" com o critério acima
df = df.withColumn("DISTANCE_TYPE", 
                   when(df['DISTANCE'] > 1000, "distant")
                   .when(df['DISTANCE'] > 500, "medium")
                   .otherwise("near"))

In [ ]:
#Analise bivariada: % atrasos por aeroporto, por regiao

In [78]:
df.printSchema()

root
 |-- col1: long (nullable = true)
 |-- col2: long (nullable = true)
 |-- col3: long (nullable = true)



In [80]:
#Removendo colunas que não serão utilizadas
df_final = df.drop('FL_DATE', 'CRS_DEP_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'CRS_ARR_TIME', 'ARR_TIME', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ARR_DELAY', 'ORIGIN', 'DEST')

In [120]:
df_final.printSchema()

root
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- Unnamed: 27: string (nullable = true)
 |-- None_dummy: integer (nullable = false)
 |-- B_dummy: integer (nullable = false)
 |-- D_dummy: integer (nullable = false)
 |-- C_dummy: integer (nullable = false)
 |-- A_dummy: integer (nullable = false)



In [121]:
# verificar quais são as colunas numéricas
colunas_numericas = [coluna for coluna, tipo in df_final.dtypes if tipo in ['int', 'double']]
# Selecionar apenas colunas numéricas
df_numericas = df_final.select(*colunas_numericas)
df_numericas.printSchema()

root
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- None_dummy: integer (nullable = false)
 |-- B_dummy: integer (nullable = false)
 |-- D_dummy: integer (nullable = false)
 |-- C_dummy: integer (nullable = false)
 |-- A_dummy: integer (nullable = false)



In [125]:
#colunas_aleat = [coluna for coluna in df_numericas.columns[:2]]
df1 = df_numericas.select('TAXI_IN', 'TAXI_OUT')
df1.printSchema()


root
 |-- TAXI_IN: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)



In [126]:
# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=df1.columns, outputCol=vector_col)
df_vector = assembler.transform(df1).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

#matrix.collect()[0]["pearson({})".format(vector_col)].values

Py4JJavaError: An error occurred while calling z:org.apache.spark.ml.stat.Correlation.corr.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 541.0 failed 1 times, most recent failure: Lost task 3.0 in stage 541.0 (TID 1223) (jupyter-flaviobs1 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$3758/0x0000000801ce5000: (struct<TAXI_IN:double,TAXI_OUT:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org

In [ ]:

#Verificar outliers
# Calcule a média e o desvio padrão
mean_value = df_numericas.agg(mean("value").alias("mean")).collect()[0]["mean"]
stddev_value = df_numericas.agg(stddev("value").alias("stddev")).collect()[0]["stddev"]

# Defina um limite para identificar outliers (por exemplo, 3 desvios padrão da média)
lower_limit = mean_value - 3 * stddev_value
upper_limit = mean_value + 3 * stddev_value

# Identifique outliers
outliers = df_numericas.filter((col("value") < lower_limit) | (col("value") > upper_limit))

# Contagem de outliers
num_outliers = outliers.count()

# Mostra o resultado
print(f"Número de outliers: {num_outliers}")
outliers.show()